In [1]:
# Decoding script, takes a GroupData set and uses Linear Discriminant Analysis
# to decode trial conditions or word tokens from neural data

import numpy as np
import os

# import matplotlib
# matplotlib.use('notebook')
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['C:\\Users\\ae166\\git\\SentenceRep_analysis'])
from analysis.grouping import GroupData
from analysis.decoding import plot_all_scores
from analysis.decoding.words import score, dict_to_structured_array

In [ ]:
# %% Imports
box = os.path.expanduser(os.path.join("~","Box"))
fpath = os.path.join(box, "CoganLab")
subjects_dir = os.path.join(box, "ECoG_Recon")
sub = GroupData.from_intermediates(
    "SentenceRep", fpath, folder='stats', subjects_dir=subjects_dir)
all_data = []
colors = [[0, 1, 0]]
scores = {'Auditory': None}
idxs = [sub.AUD]
idxs = [list(idx) for idx in idxs]
names = list(scores.keys())
conds = [['aud_ls', 'aud_lm']]
window_kwargs = {'window': 20, 'obs_axs': 1, 'normalize': 'true', 'n_jobs': -2,
                'average_repetitions': False}



Loading zscore:   6%|▌         | 2/34 [00:00<00:15,  2.08it/s]

In [ ]:
# %% Time Sliding decoding for word tokens
scores = score({'heat': 1, 'hoot': 2, 'hot': 3, 'hut': 4}, 0.8, 'lda', 5, 1, sub, idxs, conds,
                            window_kwargs, scores,
                            shuffle=False)
dict_to_structured_array(scores, 'true_scores.npy')



In [ ]:
# %% Plotting
data_dir = ''
true_scores = np.load(data_dir + 'true_scores.npy', allow_pickle=True)[0]
true_scores = {name: true_scores[name] for name in true_scores.dtype.names}

plots = {}
for key, values in scores.items():
    if values is None:
        continue
    plots[key] = np.mean(values.T[np.eye(4).astype(bool)].T, axis=2)
fig, axs = plot_all_scores(plots, conds, {n: i for n, i in zip(names, idxs)}, colors, "Word Decoding")

for ax in fig.axes:
    ax.axhline(0.25, color='k', linestyle='--')
